In [20]:
import pandas as pd
from pathlib import Path
import time

# ⏱ Iniciar temporizador
inicio = time.time()

# 📁 Ruta base
base_dir = Path(r"\\sassystem\GEIH-TEMATICA\Python\Bases_parquet")

# 🔑 Llaves de combinación
claves = ['DIRECTORIO', 'SECUENCIA_P', 'ORDEN']

# 🧩 Capítulos (se omite '01')
capitulos = {
    '10': 'generales_salud_educacion',
    '50': 'fuerza_trabajo',
    '60': 'ocupados',
    '70': 'no_ocupados',
    '80': 'otras_formas_trabajo',
    '90': 'otros_ingresos_impuestos',
    '94': 'migracion'
}

# 🎯 Años y meses a procesar (modificables)
anios = [2023]
meses = range(1, 13)

# 📦 Lista para guardar data combinada por periodo
data_por_periodo = []

for anio in anios:
    for mes in meses:
        mes_str = f"{mes:02d}"
        sufijo = f"{str(anio)[2:]}{mes_str}"  # p.ej. 2201
        carpeta = base_dir / str(anio) / mes_str / "nacional" / "dat" / "reg"
        
        print(f"\n🔄 Procesando año {anio}, mes {mes_str}...")

        data_capitulos = []

        for cod in capitulos.keys():
            archivo = carpeta / f"{cod}nal{sufijo}.parquet"

            if archivo.exists():
                try:
                    df = pd.read_parquet(str(archivo))  # requiere pyarrow o fastparquet

                    if all(col in df.columns for col in claves):
                        df = df.loc[:, ~df.columns.duplicated()]
                        data_capitulos.append(df)
                        print(f"✔ {archivo.name} cargado correctamente.")
                    else:
                        faltantes = [c for c in claves if c not in df.columns]
                        print(f"⚠ {archivo.name} omitido (faltan claves: {faltantes})")
                except Exception as e:
                    print(f"⚠ Error al leer {archivo.name}: {e}")
            else:
                print(f"❌ No encontrado: {archivo.name}")

        if data_capitulos:
            df_merge = data_capitulos[0]
            for df_next in data_capitulos[1:]:
                columnas_comunes = set(df_merge.columns).intersection(df_next.columns) - set(claves)
                if columnas_comunes:
                    df_next = df_next.drop(columns=list(columnas_comunes))
                df_merge = pd.merge(df_merge, df_next, on=claves, how='outer')

            df_merge["año"] = anio
            df_merge["mes"] = mes
            data_por_periodo.append(df_merge)
            print(f"✅ Datos combinados para {anio}-{mes_str}: {len(df_merge):,} registros")
        else:
            print(f"⚠ No se pudo combinar ningún capítulo para {anio}-{mes_str}")

# Combinar todos los periodos
if data_por_periodo:
    texto = pd.concat(data_por_periodo, ignore_index=True)
    print(f"\n✅ Base final consolidada: {len(texto):,} registros\n")
else:
    texto = pd.DataFrame()
    print("\n❌ No se cargó ningún periodo válido.")

# 📊 Resumen por año y mes
if not texto.empty:
    resumen = texto.groupby(['año', 'mes']).size().reset_index(name='registros')
    print("📋 Registros por año y mes:")
    print(resumen)

# ⏱ Tiempo total
fin = time.time()
print(f"\n⏱ Tiempo total de ejecución: {fin - inicio:.2f} segundos")


🔄 Procesando año 2023, mes 01...
✔ 10nal2301.parquet cargado correctamente.
✔ 50nal2301.parquet cargado correctamente.
✔ 60nal2301.parquet cargado correctamente.
✔ 70nal2301.parquet cargado correctamente.
✔ 80nal2301.parquet cargado correctamente.
✔ 90nal2301.parquet cargado correctamente.
✔ 94nal2301.parquet cargado correctamente.
✅ Datos combinados para 2023-01: 73,471 registros

🔄 Procesando año 2023, mes 02...
✔ 10nal2302.parquet cargado correctamente.
✔ 50nal2302.parquet cargado correctamente.
✔ 60nal2302.parquet cargado correctamente.
✔ 70nal2302.parquet cargado correctamente.
✔ 80nal2302.parquet cargado correctamente.
✔ 90nal2302.parquet cargado correctamente.
✔ 94nal2302.parquet cargado correctamente.
✅ Datos combinados para 2023-02: 74,214 registros

🔄 Procesando año 2023, mes 03...
✔ 10nal2303.parquet cargado correctamente.
✔ 50nal2303.parquet cargado correctamente.
✔ 60nal2303.parquet cargado correctamente.
✔ 70nal2303.parquet cargado correctamente.
✔ 80nal2303.parquet carg

In [21]:
texto.shape

(860802, 497)

In [22]:
texto

,PERIODO,MES,PER,DIRECTORIO,SECUENCIA_P,ORDEN,HOGAR,REGIS,AREA,CLASE,...,P3384,P3384S1,P3384S2,P3384S3,P3385,P3386,P3386S1,P3375,año,mes
0,20230104.0,01,2023.0,7309060.0,1.0,1.0,1.0,10,,1,...,2.0,NaN,NaN,NaN,1.0,NaN,,NaN,2023,1
1,20230104.0,01,2023.0,7309060.0,1.0,2.0,1.0,10,,1,...,2.0,NaN,NaN,NaN,1.0,NaN,,NaN,2023,1
2,20230104.0,01,2023.0,7309060.0,1.0,3.0,1.0,10,,1,...,2.0,NaN,NaN,NaN,1.0,NaN,,NaN,2023,1
3,20230104.0,01,2023.0,7309061.0,1.0,1.0,1.0,10,,1,...,2.0,NaN,NaN,NaN,1.0,NaN,,NaN,2023,1
4,20230104.0,01,2023.0,7309061.0,1.0,2.0,1.0,10,,1,...,2.0,NaN,NaN,NaN,1.0,NaN,,NaN,2023,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
860797,20231249.0,12,2023.0,7655975.0,1.0,4.0,1.0,10,27,1,...,2.0,,,,1.0,NaN,,NaN,2023,12
860798,20231249.0,12,2023.0,7655975.0,1.0,5.0,1.0,10,27,1,...,2.0,,,,1.0,NaN,,NaN,2023,12
860799,20231249.0,12,2023.0,7655975.0,1.0,6.0,1.0,10,27,1,...,2.0,,,,1.0,NaN,,NaN,2023,12
860800,20231249.0,12,2023.0,7655975.0,1.0,7.0,1.0,10,27,1,...,2.0,,,,1.0,NaN,,NaN,2023,12


In [3]:
texto.loc[texto['P6040'] < 15, ['OCI', 'DSI', 'FFT']] = None
texto['PET'] = (texto['P6040'] >= 15).astype(int)

texto['OCU'] = pd.NA
texto['DES'] = pd.NA
texto['INA'] = pd.NA
texto['PEA'] = pd.NA

texto.loc[texto['PET'] == 1, ['OCU', 'DES', 'INA']] = 0
texto.loc[texto['OCI'] == 1, 'OCU'] = 1
texto.loc[texto['DSI'] == 1, 'DES'] = 1
texto.loc[texto['FFT'] == 1, 'INA'] = 1
texto.loc[texto['PET'] == 1, 'PEA'] = 0
texto.loc[(texto['OCU'] == 1) | (texto['DES'] == 1), 'PEA'] = 1

texto['fex1'] = texto['FEX_C18'] / 1000
texto['fex12'] = texto['FEX_C18'] / 12000

texto['sexo'] = texto['P3271'].map({1: 'Hombre', 2: 'Mujer'})

In [4]:
""""                      
VALIDAR PEGUE             
"""                       

df_tab = texto[['mes', 'OCU', 'fex1']].dropna(subset=['OCU', 'fex1'])

df_tab['ocu_ponderado'] = df_tab['OCU'] * df_tab['fex1']

resumen = df_tab.groupby('mes')['ocu_ponderado'].sum().reset_index()

total = pd.DataFrame([{
    'mes': 'Total',
    'ocu_ponderado': resumen['ocu_ponderado'].sum()
}])

resultado_final = pd.concat([resumen, total], ignore_index=True)

resultado_final['ocu_ponderado'] = resultado_final['ocu_ponderado'].map('{:,.3f}'.format)

print(resultado_final)

      mes ocu_ponderado
0       1    21,491.566
1       2    22,232.690
2       3    22,793.699
3       4    22,741.957
4       5    22,567.452
5       6    23,052.256
6       7    23,181.793
7       8    23,161.076
8       9    23,106.591
9      10    23,081.781
10     11    23,185.541
11     12    22,864.323
12  Total   273,460.725


In [19]:
import pandas as pd
import numpy as np
from typing import List, Optional, Callable
import pandas.io.formats.style

def proc_tabulate(
    df: pd.DataFrame,
    class_rows: List[str],
    class_cols: List[str],
    var_list: List[str],
    stats: List[str] = ['mean', 'sum'],
    weight: Optional[str] = None,
    format_func: Optional[Callable] = None,
    include_missing: bool = False
) -> pandas.io.formats.style.Styler:
    """
    Emula PROC TABULATE de SAS con visualización en Python.

    Args:
        df (pd.DataFrame): DataFrame de entrada.
        class_rows (list): Variables categóricas en las filas.
        class_cols (list): Variables categóricas en las columnas.
        var_list (list): Lista de variables numéricas.
        stats (list): Lista de estadísticas (ej. ['mean', 'sum']).
        weight (str): Variable de pesos (opcional).
        format_func (callable): Función para formatear los valores.
        include_missing (bool): Incluir faltantes como categoría.

    Returns:
        pd.io.formats.style.Styler: Visualización estilo SAS.
    """
    df = df.copy()

    # Incluir valores faltantes como categoría
    if include_missing:
        for col in class_rows + class_cols:
            df[col] = df[col].astype("category").cat.add_categories("Missing").fillna("Missing")

    results = []

    for var in var_list:
        working_df = df.copy()

        if weight:
            for stat in stats:
                if stat == 'mean':
                    working_df['_weighted'] = working_df[var] * working_df[weight]
                    pivot = pd.pivot_table(
                        working_df,
                        values=['_weighted', weight],
                        index=class_rows,
                        columns=class_cols,
                        aggfunc='sum',
                        dropna=not include_missing
                    )
                    weighted_mean = pivot['_weighted'] / pivot[weight]
                    weighted_mean.columns = [f"{var}_mean_{col}" for col in weighted_mean.columns]
                    results.append(weighted_mean)

                elif stat == 'sum':
                    working_df['_weighted'] = working_df[var] * working_df[weight]
                    weighted_sum = pd.pivot_table(
                        working_df,
                        values='_weighted',
                        index=class_rows,
                        columns=class_cols,
                        aggfunc='sum',
                        dropna=not include_missing
                    )
                    weighted_sum.columns = [f"{var}_sum_{col}" for col in weighted_sum.columns]
                    results.append(weighted_sum)

                elif stat == 'count':
                    count = pd.pivot_table(
                        working_df,
                        values=var,
                        index=class_rows,
                        columns=class_cols,
                        aggfunc='count',
                        dropna=not include_missing
                    )
                    count.columns = [f"{var}_count_{col}" for col in count.columns]
                    results.append(count)

        else:
            pivot = pd.pivot_table(
                working_df,
                values=var,
                index=class_rows,
                columns=class_cols,
                aggfunc=stats,
                dropna=not include_missing
            )
            pivot.columns = [f"{var}_{stat}_{col}" for stat, col in pivot.columns]
            results.append(pivot)

    final_result = pd.concat(results, axis=1)

    if format_func:
        styled = final_result.style.format(format_func)
    else:
        styled = final_result.style

    styled.set_caption("Tabla tipo PROC TABULATE (SAS replicado en Python)")
    styled.set_table_styles(
        [{'selector': 'caption', 'props': [('caption-side', 'top'), ('font-weight', 'bold')]}]
    )

    return styled

In [13]:
texto.columns

Index(['PERIODO', 'MES', 'PER', 'DIRECTORIO', 'SECUENCIA_P', 'ORDEN', 'HOGAR',
       'REGIS', 'AREA', 'CLASE',
       ...
       'P3386S1', 'P3375', 'año', 'mes', 'OCU', 'DES', 'INA', 'PEA', 'fex1',
       'fex12'],
      dtype='object', length=503)

In [6]:
columnas_necesarias = ['P3271', 'año', 'AREA', 'fex1','fex12','MES','OCU','P3271','sexo']
df_reducido = texto[columnas_necesarias].copy()
df_reducido

,P3271,año,AREA,fex1,fex12,MES,OCU,P3271,sexo
0,2.0,2023,,1.580677,0.131723,01,0,2.0,Mujer
1,1.0,2023,,1.580677,0.131723,01,1,1.0,Hombre
2,2.0,2023,,1.580677,0.131723,01,<NA>,2.0,Mujer
3,2.0,2023,,1.001661,0.083472,01,1,2.0,Mujer
4,1.0,2023,,1.001661,0.083472,01,1,1.0,Hombre
...,...,...,...,...,...,...,...,...,...
860797,2.0,2023,27,0.044982,0.003749,12,0,2.0,Mujer
860798,2.0,2023,27,0.044982,0.003749,12,1,2.0,Mujer
860799,1.0,2023,27,0.044982,0.003749,12,0,1.0,Hombre
860800,1.0,2023,27,0.044982,0.003749,12,<NA>,1.0,Hombre


In [7]:
# Paso 1: Asegúrate de tener las columnas necesarias y en el tipo correcto
df_reducido['AREA'] = df_reducido['AREA'].astype('category')
df_reducido['año'] = df_reducido['año'].astype('category')
df_reducido['MES'] = df_reducido['MES'].astype('category')
df_reducido['sexo'] = df_reducido['sexo'].astype('category')

In [18]:
tabla = proc_tabulate_python_mdo(
    df=df_reducido,
    class_rows=['sexo'],               
    class_cols=['año'],         
    var_list=['OCU'],                  
    stats=['sum'],                     
    weight='fex12',                     
    format_func=lambda x: f"{x:,.3f}", 
    include_missing=True               
)

tabla


,OCU_sum_2023,OCU_sum_Missing
sexo,,
Hombre,"13,356.180",0.000
Mujer,"9,432.214",0.000
Missing,0.000,0.000


In [ ]:
hgjh